<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [ ]:
url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
## company
broswer_path = r"C:\Users\superuser\chromedriver-win64\chromedriver.exe"
## local
# broswer_path = r"C:\Users\yunjc\chromedriver_win32\chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(broswer_path, chrome_options=options)

In [ ]:
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))

def parsing_laws():
    lang_pairs = {"eng": [], "kor": []}
    for i, p in enumerate(driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')):
        if i<4: continue
        for line in p.text.split("\n"):
            if i%2==0:
                lang_pairs["eng"].append(line)
            else:
                lang_pairs["kor"].append(line)

In [ ]:
driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()
